In [13]:
import warnings
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import ast
import models
import eventstox

%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
df_1819 = pd.read_csv('df_1819.csv')
df_1920 = pd.read_csv('df_1920.csv')
df_2021 = pd.read_csv('df_2021.csv')

X_1819, y_1819 = eventstox.df_to_X_y(df_1819)
X_1920, y_1920 = eventstox.df_to_X_y(df_1920)
X_2021, y_2021 = eventstox.df_to_X_y(df_2021)

In [85]:
from sklearn.preprocessing import MinMaxScaler

def preprocessing(X, y):

    numeric_features = [
        f"{feature}_{i}" for i in range(10) for feature in [
        'location_x', 'location_y', 'end_location_x', 'end_location_y'
        ]
        ]
    X_numeric = X[numeric_features]

    binary_features = [f"{feature}_{i}" for i in range(10) for feature in [
        'outcome', 'team']]
    X_binary = X[binary_features]

    scaler = MinMaxScaler()
    X_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(X_numeric), columns=X_numeric.columns)

    X_scaled = pd.concat([X_numeric_scaled, X_binary], axis=1)


    data = pd.concat([X_scaled, pd.DataFrame({'y': y})], axis=1)
    data_cleaned = data.dropna()

    X_cleaned = data_cleaned.drop('y', axis=1)
    y_cleaned = data_cleaned['y']

    return X_cleaned, y_cleaned


X = pd.concat([X_1819, X_1920], axis=0).reset_index(drop=True)
y = np.concatenate([y_1819, y_1920], axis=0)

X_scaled, y = preprocessing(X, y)

In [90]:
features = ['location_x', 'location_y', 'end_location_x',
            'end_location_y', 'outcome', 'team']
feature_columns = [
    f"{feature}_{i}" for i in range(10)
    for feature in features
]


def get_lstm_arr(X: pd.DataFrame):

    # Extract features for each timestamp
    lstm_arr = X[feature_columns].to_numpy().reshape(
        len(X), 10, len(features)
    )

    return lstm_arr

X_lstm = get_lstm_arr(X)